In [1]:
import pandas as pd
import requests

from job_apps.constants import API_HEADERS, URL_DATABASE

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
search_response = requests.post(url=URL_DATABASE, headers=API_HEADERS)
search_response_dict = search_response.json()

In [17]:
has_more = True
cursor = None
results = []
body = {}

while has_more:
    if not cursor:
        search_response = requests.post(url, headers=headers)
    else:
        search_response = requests.post(url, headers=headers, json=body)

    search_response_dict = search_response.json()
    has_more = search_response_dict["has_more"]
    cursor = search_response_dict["next_cursor"]
    body = {"start_cursor": cursor}

    results_loc = search_response_dict["results"]
    results += results_loc

In [18]:
def get_dict(result):
    if result["properties"]["Company"]["rich_text"]:
        company = result["properties"]["Company"]["rich_text"][0]["text"]["content"]
    else:
        company = ''

    if result["properties"]["Job Title"]["title"]:
        job_title = result["properties"]["Job Title"]["title"][0]["text"]["content"]
    else:
        job_title = ''

    if result["properties"]["Date Applied"]["date"]:
        date_applied = result["properties"]["Date Applied"]["date"]["start"]
    else:
        date_applied = ''

    if result["properties"]["Origin"]["select"]:
        origin = result["properties"]["Origin"]["select"]["name"]
    else:
        origin = ''

    if result["properties"]["Stage"]["select"]:
        stage = result["properties"]["Stage"]["select"]["name"]
    else:
        stage = ''

    if result["properties"]["Job Description"]["rich_text"]:
        job_description = result["properties"]["Job Description"]["rich_text"][0]["plain_text"]
    else:
        job_description = ''

    if result["properties"]['Cover letter']['select']:
        cover_letter = result["properties"]['Cover letter']['select']['name']
    else:
        cover_letter = ''
        
    result_dict = {"Job Title": job_title,
               "Company": company,
               "Date Applied": date_applied,
               "Origin": origin,
               "Stage": stage,
               "Job Description": job_description,
               'Cover letter': cover_letter}
    return result_dict

In [19]:
entries = []

for result in results:
    entry = get_dict(result)
    entries.append(entry)

df = pd.DataFrame(entries)

In [20]:
df

,Job Title,Company,Date Applied,Origin,Stage,Job Description,Cover letter
0,Data Scientist,Groupe BPCE,2024-09-11,Welcome To The Jungle,Application Sent,https://www.welcometothejungle.com/en/companie...,Letter
1,Post docteur Datascience,Finovox,2024-09-11,Welcome To The Jungle,Application Sent,https://www.welcometothejungle.com/en/companie...,Letter
2,Data Scientist,Gameloft,2024-09-11,Welcome To The Jungle,Rejection,https://www.welcometothejungle.com/en/companie...,Letter
3,Data Scientist - Timesseries Machine Learning ...,Schneider Electric,2024-09-11,Welcome To The Jungle,Application Sent,https://www.welcometothejungle.com/en/companie...,Letter
4,Consultant Data / IA,Noveane,2024-09-10,Welcome To The Jungle,Application Sent,https://www.welcometothejungle.com/en/companie...,Letter
...,...,...,...,...,...,...,...
300,Data Analyst,Societe Generale,2024-01-05,LinkedIn,Rejection,https://www.linkedin.com/jobs/view/3798999621/...,
301,"Research Scientist Intern, AI Core Machine Lea...",Meta,2024-01-05,LinkedIn,Application Sent,https://www.linkedin.com/jobs/view/3798417343/...,
302,Développeur scientifique,ASTEK,2024-01-05,LinkedIn,Application Sent,https://www.linkedin.com/jobs/view/3693935151/...,
303,Ingénieur-e de recherche data science,Saint-Gobain,2024-01-04,LinkedIn,Application Sent,https://joinus.saint-gobain.com/fr/fra/red/p/6...,


In [27]:
df['Stage'].value_counts(normalize=True).loc['Application Sent'] + df['Stage'].value_counts(normalize=True).loc['Rejection']

np.float64(0.9770491803278689)